In [3]:
import opensim as osim
import json
import numpy as np
from math import pi, atan2, sqrt, acos, sin, cos

In [ ]:
def load_and_analyze_model(model_path):
    """
    Load OpenSim model and extract all important information
    
    Args:
        model_path (str): Path to .osim model file
    
    Returns:
        dict: Model information
    """
    try:
        # Load the model
        print(f"Loading model: {model_path}")
        model = osim.Model(model_path)
        
        # Initialize the system (important step!)
        state = model.initSystem()
        
        print(f"✓ Model loaded successfully: {model.getName()}")
        print(f"✓ Model description: {model.getDescription()}")
        print("=" * 50)
        
        # Get model information
        model_info = {
            'name': model.getName(),
            'description': model.getDescription(),
            'coordinates': [],
            'bodies': [],
            'joints': [],
            'muscles': []
        }
        
        # 1. COORDINATES (joint angles/positions)
        print("\n COORDINATES (Joint Angles):")
        coord_set = model.getCoordinateSet()
        print(f"Total coordinates: {coord_set.getSize()}")
        
        for i in range(coord_set.getSize()):
            coord = coord_set.get(i)
            coord_info = {
                'name': coord.getName(),
                'motion_type': coord.getMotionType().name if hasattr(coord.getMotionType(), 'name') else str(coord.getMotionType()),
                'default_value': coord.getDefaultValue(),
                'range_min': coord.getRangeMin(),
                'range_max': coord.getRangeMax(),
                'locked': coord.getDefaultLocked()
            }
            model_info['coordinates'].append(coord_info)
            
            print(f"  {i+1}. {coord.getName()}")
            print(f"     Type: {coord_info['motion_type']}")
            print(f"     Range: [{coord.getRangeMin():.3f}, {coord.getRangeMax():.3f}]")
            print(f"     Default: {coord.getDefaultValue():.3f}")
            print(f"     Locked: {coord.getDefaultLocked()}")
        
        # 2. BODIES (rigid body segments)
        print(f"\n BODIES (Rigid Segments):")
        body_set = model.getBodySet()
        print(f"Total bodies: {body_set.getSize()}")
        
        for i in range(body_set.getSize()):
            body = body_set.get(i)
            body_info = {
                'name': body.getName(),
                'mass': body.getMass()
            }
            model_info['bodies'].append(body_info)
            
            print(f"  {i+1}. {body.getName()}")
            print(f"     Mass: {body.getMass():.3f} kg")
        
        # 3. JOINTS (connections between bodies)
        print(f"\n🔗 JOINTS (Connections):")
        joint_set = model.getJointSet()
        print(f"Total joints: {joint_set.getSize()}")
        
        for i in range(joint_set.getSize()):
            joint = joint_set.get(i)
            joint_info = {
                'name': joint.getName(),
                'type': joint.getConcreteClassName(),
                'parent_body': joint.getParentFrame().getName(),
                'child_body': joint.getChildFrame().getName()
            }
            model_info['joints'].append(joint_info)
            
            print(f"  {i+1}. {joint.getName()}")
            print(f"     Type: {joint.getConcreteClassName()}")
            print(f"     Connects: {joint.getParentFrame().getName()} → {joint.getChildFrame().getName()}")
        
        # 4. MUSCLES (if any)
        print(f"\n MUSCLES:")
        muscle_set = model.getMuscles()
        print(f"Total muscles: {muscle_set.getSize()}")
        
        if muscle_set.getSize() > 0:
            for i in range(min(5, muscle_set.getSize())):  # Show first 5 only
                muscle = muscle_set.get(i)
                muscle_info = {
                    'name': muscle.getName(),
                    'max_force': muscle.getMaxIsometricForce()
                }
                model_info['muscles'].append(muscle_info)
                
                print(f"  {i+1}. {muscle.getName()}")
                print(f"     Max Force: {muscle.getMaxIsometricForce():.1f} N")
            
            if muscle_set.getSize() > 5:
                print(f"  ... and {muscle_set.getSize() - 5} more muscles")
        else:
            print("No muscles in this model")
        
        return model_info, model, state
        
    except Exception as e:
        print(f"Error loading model: {e}")
        return None, None, None

In [16]:
model_path = "W:/Models/Models/Arm26/arm26.osim"  # Replace with your model path
load_and_analyze_model(model_path)

Loading model: W:/Models/Models/Arm26/arm26.osim
✓ Model loaded successfully: arm26
✓ Model description: 

📐 COORDINATES (Joint Angles):
Total coordinates: 2
  1. r_shoulder_elev
     Type: 1
     Range: [-1.571, 3.142]
     Default: 0.000
     Locked: False
  2. r_elbow_flex
     Type: 1
     Range: [0.000, 2.269]
     Default: 0.000
     Locked: False

🦴 BODIES (Rigid Segments):
Total bodies: 2
  1. r_humerus
     Mass: 1.865 kg
  2. r_ulna_radius_hand
     Mass: 1.534 kg

🔗 JOINTS (Connections):
Total joints: 2
  1. r_shoulder
     Type: CustomJoint
     Connects: ground_offset → r_humerus_offset
  2. r_elbow
     Type: CustomJoint
     Connects: r_humerus_offset → r_ulna_radius_hand_offset

💪 MUSCLES:
Total muscles: 6
  1. TRIlong
     Max Force: 798.5 N
  2. TRIlat
     Max Force: 624.3 N
  3. TRImed
     Max Force: 624.3 N
  4. BIClong
     Max Force: 624.3 N
  5. BICshort
     Max Force: 435.6 N
  ... and 1 more muscles


({'name': 'arm26',
  'description': '',
  'coordinates': [{'name': 'r_shoulder_elev',
    'motion_type': '1',
    'default_value': 0.0,
    'range_min': -1.57079633,
    'range_max': 3.14159265,
    'locked': False},
   {'name': 'r_elbow_flex',
    'motion_type': '1',
    'default_value': 0.0,
    'range_min': 0.0,
    'range_max': 2.26892803,
    'locked': False}],
  'bodies': [{'name': 'r_humerus', 'mass': 1.864572},
   {'name': 'r_ulna_radius_hand', 'mass': 1.534315}],
  'joints': [{'name': 'r_shoulder',
    'type': 'CustomJoint',
    'parent_body': 'ground_offset',
    'child_body': 'r_humerus_offset'},
   {'name': 'r_elbow',
    'type': 'CustomJoint',
    'parent_body': 'r_humerus_offset',
    'child_body': 'r_ulna_radius_hand_offset'}],
  'muscles': [{'name': 'TRIlong', 'max_force': 798.52},
   {'name': 'TRIlat', 'max_force': 624.3},
   {'name': 'TRImed', 'max_force': 624.3},
   {'name': 'BIClong', 'max_force': 624.3},
   {'name': 'BICshort', 'max_force': 435.56}]},
 <opensim.sim

In [10]:
import opensim as osim

def connect_to_opensim(model_path, trajectory):
    """
    Connect our trajectory to an OpenSim arm model
    """
    try:
        # Load OpenSim model
        model = osim.Model(model_path)
        state = model.initSystem()
        
        # Get coordinate references
        coord_set = model.getCoordinateSet()
        shoulder_coord = coord_set.get('shoulder_elv')  # Adjust names as needed
        elbow_coord = coord_set.get('elbow_flexion')    # Based on your model
        
        # Create a TimeSeriesTable for the motion
        time = trajectory['time']
        labels = osim.StdVectorString()
        labels.append('shoulder_elv')
        labels.append('elbow_flexion')
        
        table = osim.TimeSeriesTable()
        for i, t in enumerate(time):
            row = osim.RowVector(
                [trajectory['theta1']['pos'][i], 
                 trajectory['theta2']['pos'][i]],
                labels)
            table.appendRow(t, row)
        
        # Add metadata about the columns
        table.addTableMetaDataString('inDegrees', 'no')
        
        # Visualize in OpenSim
        model.updDisplayHints().show_markers(True)
        viz = osim.Visualizer(model)
        viz.setShowSimulation(True)
        
        for i, t in enumerate(time):
            state.setTime(t)
            shoulder_coord.setValue(state, trajectory['theta1']['pos'][i])
            elbow_coord.setValue(state, trajectory['theta2']['pos'][i])
            model.realizePosition(state)
            viz.draw(state)
            time.sleep(0.01)  # Slow down visualization
        
        return table
        
    except Exception as e:
        print(f"Error connecting to OpenSim: {e}")
        return None

# Example usage (adjust model path and coordinate names)
# motion_table = connect_to_opensim('arm_model.osim', trajectory)

creating a 3 link arm for 3D kinematics implementation


In [8]:
class ThreeLinkArm:
    def __init__(self, L1=0.3, L2=0.25, L3=0.2):
        """Initialize the three-link arm with given segment lengths.
        Args:
            L1: Length of the shoulder to elbow segment
            L2: Length of the elbow to the wrist segment
            L3: Length of the wrist to the hand segment
        """
        self.L1 = L1
        self.L2 = L2
        self.L3 = L3
        self.joint_limitis = {
            'shoulder_elv': (-pi/2, pi/2),
            'elbow_rot': (-pi, pi),
            'elbow_flex': (0, 5*pi/6),
            'wrist_flex': (-pi/2, pi/2)
        }

    def inverse_kinematics(self, target_position, target_orientation, initial_angle):
        """
        Calculate joint angles for 3D reaching.
        Args:
            target_position (x, y, z):
                x: X-coordinate of the end-effector
                y: Y-coordinate of the end-effector
                z: Z-coordinate of the end-effector
        Returns:
            Dict: Joint angles (shoulder_elv, elbow_rot, elbow_flex, wrist_flex)
        """

        x,y,z = target_position
        solutions = []

        # Solve for shoulder elevation angle and rotation in spherical coordinates
        r_xy = sqrt(x**2 + y**2)
        r = sqrt(r_xy**2 + z**2)

        # A check to ensure the target is reachable
        if r > (self.L1 + self.L2 + self.L3) or r < abs(self.L1 - self.L2 - self.L3):
            raise ValueError("Target position is out of reach for the arm.")
        
        # Shoulder rotation angle about the z axis
        shoulder_rot = atan2(y, x)
        # Shoulder elevation angle about the y axis
        shoulder_elv = atan2(z, r_xy)

        # Solve for 2D IK in the arm plane
        # Traget position in the arm plane
        planar_target = r - self.L3, 0 # to Simplify calculations Wrist Alignment: The hand (L₃) will always point directly at the target
        """
        Precise Wrist Orentation:
        # More accurate version would include wrist orientation
        planar_target = sqrt(r_xy**2 + z**2) - L3*cos(wrist_angle), L3*sin(wrist_angle)
        
        Target Avoidance:
        # Would need to compute the actual 3D elbow position
        elbow_pos = shoulder_pos + L1*direction_vector
        """

        try: 
            elbow_up_sol = self.solve_2d_ik(planar_target[0], planar_target[1], elbow_up=True)
            elbow_down_sol = self.solve_2d_ik(planar_target[0], planar_target[1], elbow_up=False)

            for sol in [elbow_up_sol, elbow_down_sol]:
                if sol is not None:
                    shoulder_elv, elbow_rot, elbow_flex, wrist_flex = sol
                    # Check joint limits
                    if (self.joint_limitis['shoulder_elv'][0] <= shoulder_elv <= self.joint_limitis['shoulder_elv'][1] and
                        self.joint_limitis['elbow_rot'][0] <= elbow_rot <= self.joint_limitis['elbow_rot'][1] and
                        self.joint_limitis['elbow_flex'][0] <= elbow_flex <= self.joint_limitis['elbow_flex'][1] and
                        self.joint_limitis['wrist_flex'][0] <= wrist_flex <= self.joint_limitis['wrist_flex'][1]):
                        solutions.append({
                            'shoulder_elv': shoulder_elv,
                            'elbow_rot': elbow_rot,
                            'elbow_flex': elbow_flex,
                            'wrist_flex': 0 # Simplified wrist flexion
                        })
        except ValueError:
            pass

        if not solutions:
            raise ValueError("No valid solutions found within joint limits.")
        
        # Return the first valid solution closest to the intial angles if provided
        if initial_angles:
            solutions.sort(key=lambda x: sum(abs(sol[angle] - initial_angles[angle]) for angle in initial_angles))

        return solutions[0]
    
    def solve_2d_ik(self, x, y, elbow_up=True):
        """
        Solve 2D inverse kinematics for the elbow joint.
        Args:
            x: X-coordinate of the target position in the arm plane
            y: Y-coordinate of the target position in the arm plane
            elbow_up: If True, solve for elbow up configuration, else elbow down
        Returns:
            Tuple: (shoulder_elv, elbow_rot, elbow_flex, wrist_flex)
        """
        # Calculate distance to target
        d = sqrt(x**2 + y**2 - self.L1**2 - self.L2**2) / (2 * self.L1 * self.L2)
        
        if abs(d) > 1:
            raise ValueError("Target position is unreachable.")
        
        theta2 = acos(d) if elbow_up else -acos(d)
        theta1 = atan2(y, x) - atan2(self.L2 * sin(theta2), self.L1 + self.L2 * cos(theta2))

        return theta1,theta2
    
    def _check_joint_limits(self, angles):
        """
        Check if the given joint angles are within the defined limits.
        Args:
            angles (dict): Joint angles to check
        Returns:
            bool: True if all angles are within limits, False otherwise
        """
        for joint, angle in angles.items():
            if joint in self.joint_limitis:
                min_limit, max_limit = self.joint_limitis[joint]
                if not (min_limit <= angle <= max_limit):
                    return False
        return True
    
    def _solution_distance(self, sol1, sol2):
        """
        Calculate the distance between two solutions based on joint angles.
        Args:
            sol1, sol2 (dict): Solutions to compare
        Returns:
            float: Distance between the two solutions
        """
        return sum(abs(sol1[joint] - sol2[joint]) for joint in sol1)


3D trajcetory Generation

In [4]:
def generate_3d_trajectory(start_pos, end_pos, duration=1.0, dt=0.01):
    """
    Generate 3D minimum jerk trajectory
    Returns: Dictionary with time, positions, velocities, accelerations
    """
    t = np.arange(0, duration + dt, dt)
    t_norm = t / duration  # Normalize time to [0, 1]
    trajectory = {
        'time':t,
        'positions': np.zeros((len(t), 3)),
        'velocities': np.zeros((len(t), 3)),
        'accelerations': np.zeros((len(t), 3))
    }
    for i in range(3):
        satrt = start_pos[i]
        end = end_pos[i]
        diff = end - satrt

        # Position
        trajectory['positions'][:, i] = satrt + diff * (10 * t_norm**3 - 15 * t_norm**4 + 6 * t_norm**5)

        # Velocity
        trajectory['velocities'][:, i] = diff * (30 * t_norm**2 - 60 * t_norm**3 + 30 * t_norm**4) / duration

        # Acceleration
        trajectory['accelerations'][:, i] = diff * (60 * t_norm - 180 * t_norm**2 + 120 * t_norm**3) / (duration**2)

    return trajectory
    

In [ ]:
model_path = "W:\OpenSim 4.5\Resources\Models\Arm26\Arm26.osim"
model = osim.Model(model_path)
state = load_and_analyze_model(model_path)

# Connect to OpenSim (replace with your model path)
model_path = "arm_model.osim"
motion_table = connect_to_opensim(model_path, trajectory)

# Export motion file
export_motion_file(trajectory, 'reaching.mot', ['shoulder_elv', 'elbow_flexion'])

# Initialize 3D arm
arm_3d = ThreeLinkArm()

# Define and execute 3D movement
start_3d = (0.3, 0.0, 0.2)
end_3d = (0.1, 0.3, 0.4)
traj_3d = generate_3d_trajectory(start_3d, end_3d)

# Solve IK for 3D movement
angles_sequence = []
current_angles = {'shoulder_elv': 0, 'shoulder_rot': 0, 'elbow_flex': pi/2, 'wrist_flex': 0}

for pos in traj_3d['positions']:
    current_angles = arm_3d.inverse_kinematics(pos, initial_angles=current_angles)
    angles_sequence.append(current_angles)




Loading model: W:\OpenSim 4.5\Resources\Models\Arm26\Arm26.osim
✓ Model loaded successfully: arm26
✓ Model description: 

📐 COORDINATES (Joint Angles):
Total coordinates: 2
  1. r_shoulder_elev
     Type: 1
     Range: [-1.571, 3.142]
     Default: 0.000
     Locked: False
  2. r_elbow_flex
     Type: 1
     Range: [0.000, 2.269]
     Default: 0.000
     Locked: False

🦴 BODIES (Rigid Segments):
Total bodies: 3
  1. base
     Mass: 0.000 kg
  2. r_humerus
     Mass: 1.865 kg
  3. r_ulna_radius_hand
     Mass: 1.534 kg

🔗 JOINTS (Connections):
Total joints: 3
  1. offset
     Type: WeldJoint
     Connects: ground_offset → base_offset
  2. r_shoulder
     Type: CustomJoint
     Connects: base_offset → r_humerus_offset
  3. r_elbow
     Type: CustomJoint
     Connects: r_humerus_offset → r_ulna_radius_hand_offset

💪 MUSCLES:
Total muscles: 6
  1. TRIlong
     Max Force: 798.5 N
  2. TRIlat
     Max Force: 624.3 N
  3. TRImed
     Max Force: 624.3 N
  4. BIClong
     Max Force: 624.3 N
  5

In [ ]:
# Initialize 3D arm
arm_3d = ThreeLinkArm()



# Define movement
start_pos = (0.3, 0.0, 0.2)
end_pos = (0.1, 0.3, 0.4)

# Generate trajectory
traj_3d = generate_3d_trajectory(start_pos, end_pos)

# Solve IK for each point
angles_sequence = []
current_angles = {
    'shoulder_elv': 0,
    'shoulder_rot': 0,
    'elbow_flex': pi/2,
    'wrist_flex': 0
}

for pos in traj_3d['positions']:
    try:
        current_angles = arm_3d.inverse_kinematics(pos, initial_angles=current_angles)
        angles_sequence.append(current_angles)
    except ValueError:
        print(f"Could not reach position {pos}")
        break



# Export to OpenSim motion file
def export_3d_motion(angles_sequence, time, filename):
    with open(filename, 'w') as f:
        f.write(f"name {filename}\n")
        f.write("datacolumns 5\n")
        f.write(f"datarows {len(time)}\n")
        f.write(f"range {time[0]} {time[-1]}\n")
        f.write("endheader\n")
        f.write("time\tshoulder_elv\tshoulder_rot\telbow_flex\twrist_flex\n")
        
        for t, angles in zip(time, angles_sequence):
            f.write(f"{t:.4f}\t{angles['shoulder_elv']:.6f}\t")
            f.write(f"{angles['shoulder_rot']:.6f}\t")
            f.write(f"{angles['elbow_flex']:.6f}\t")
            f.write(f"{angles['wrist_flex']:.6f}\n")

export_3d_motion(angles_sequence, traj_3d['time'], '3d_reaching.mot')

In [ ]:
import numpy as np
import opensim as osim
from math import pi, atan2, acos, sqrt

class ThreeLinkArm:
    def __init__(self, L1=0.3, L2=0.25, L3=0.2):
        self.L1 = L1  # Upper arm
        self.L2 = L2  # Forearm
        self.L3 = L3  # Hand
        self.joint_limits = {
            'shoulder_elv': (-pi/2, pi/2),  # -90° to 90°
            'shoulder_rot': (-pi, pi),      # -180° to 180°
            'elbow_flex': (0, 5*pi/6)       # 0° to 150°
        }

    def inverse_kinematics(self, target_pos, initial_angles=None):
        """Compute 3D joint angles for (x,y,z) target"""
        x, y, z = target_pos
        r_xy = sqrt(x**2 + y**2)
        r = sqrt(r_xy**2 + z**2)
        
        # Check reachability
        if r > (self.L1 + self.L2 + self.L3):
            raise ValueError("Target unreachable")
        
        # Shoulder rotation (yaw)
        shoulder_rot = atan2(y, x)
        
        # Shoulder elevation (pitch)
        shoulder_elv = atan2(z, r_xy)
        
        # Elbow flexion (reuse 2D IK in the arm plane)
        planar_target = r - self.L3, 0
        D = (planar_target[0]**2 - self.L1**2 - self.L2**2) / (2 * self.L1 * self.L2)
        elbow_flex = acos(np.clip(D, -1, 1))
        
        return {
            'shoulder_elv': shoulder_elv,
            'shoulder_rot': shoulder_rot,
            'elbow_flex': elbow_flex
        }

def generate_3d_trajectory(start_pos, end_pos, duration=1.0, dt=0.01):
    """Minimum-jerk trajectory in 3D"""
    t = np.arange(0, duration + dt, dt)
    t_norm = t / duration
    trajectory = np.zeros((len(t), 3))
    
    for i in range(3):  # x,y,z
        diff = end_pos[i] - start_pos[i]
        trajectory[:, i] = start_pos[i] + diff * (10*t_norm**3 - 15*t_norm**4 + 6*t_norm**5)
    
    return trajectory, t

def export_to_opensim(model_path, angles_sequence, time):
    """Send motion data to OpenSim"""
    model = osim.Model(model_path)
    state = model.initSystem()
    
    # Map our angles to OpenSim coordinates
    coord_set = model.getCoordinateSet()
    shoulder_elv = coord_set.get("shoulder_elv")  # Adjust names to match your model!
    shoulder_rot = coord_set.get("shoulder_rot") 
    elbow_flex = coord_set.get("elbow_flex")
    
    # Visualize in OpenSim
    viz = osim.Visualizer(model)
    viz.setShowSimulation(True)
    
    for i, t in enumerate(time):
        state.setTime(t)
        shoulder_elv.setValue(state, angles_sequence[i]['shoulder_elv'])
        shoulder_rot.setValue(state, angles_sequence[i]['shoulder_rot'])
        elbow_flex.setValue(state, angles_sequence[i]['elbow_flex'])
        model.realizePosition(state)
        viz.draw(state)
        osim.opensim.common.sleep(0.01)

# Example Usage
if __name__ == "__main__":
    # 1. Initialize arm and trajectory
    arm = ThreeLinkArm()
    start_pos = (0.3, 0.0, 0.2)
    end_pos = (0.1, 0.3, 0.4)
    trajectory, time = generate_3d_trajectory(start_pos, end_pos)
    
    # 2. Solve IK for each point
    angles_sequence = []
    for pos in trajectory:
        angles = arm.inverse_kinematics(pos)
        angles_sequence.append(angles)
    
    # 3. Connect to OpenSim (replace with your model path)
    export_to_opensim("arm_model.osim", angles_sequence, time)

In [ ]:
import opensim as osim
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

# ======================== FORWARD KINEMATICS ========================
def get_hand_position(model, state, body_name="hand"):
    """ Calculate hand position in global coordinates """
    model.realizePosition(state)
    body = model.getBodySet().get(body_name)
    position = body.getPositionInGround(state)
    return np.array([position.get(0), position.get(1), position.get(2)])

# ====================== INVERSE KINEMATICS ========================
def setup_inverse_kinematics(model, target_body_name, target_positions, times):
    """ Configure IK to track desired hand trajectory """
    ik_tool = osim.InverseKinematicsTool()
    ik_tool.setModel(model)
    ik_tool.setStartTime(times[0])
    ik_tool.setEndTime(times[-1])
    
    # Create and add markers
    marker_set = osim.MarkerSet()
    
    # Target marker (what we want to reach)
    target_marker = osim.Marker("target_marker", 
                              model.getGround(), 
                              osim.Vec3(0, 0, 0))
    marker_set.cloneAndAppend(target_marker)
    
    # Hand marker (end effector)
    hand_marker = osim.Marker("hand_marker",
                            model.getBodySet().get(target_body_name),
                            osim.Vec3(0, 0, 0))
    marker_set.cloneAndAppend(hand_marker)
    
    model.updateMarkerSet(marker_set)
    
    # Create marker data table
    marker_data = osim.TimeSeriesTableVec3()
    for t in times:
        marker_data.appendRow(t, osim.RowVectorVec3(2, osim.Vec3(0)))
    
    # Fill with target positions
    for i in range(len(times)):
        marker_data.setRowAtIndex(i, osim.RowVectorVec3([
            osim.Vec3(*target_positions[i]),  # Target position
            osim.Vec3(0, 0, 0)  # Hand marker position (local coords)
        ]))
    
    osim.STOFileAdapterVec3.write(marker_data, "ik_marker_data.trc")
    
    # Configure IK tool
    ik_tool.setMarkerDataFileName("ik_marker_data.trc")
    ik_tool.setOutputMotionFileName("ik_results.mot")
    ik_tool.set_report_errors(True)
    ik_tool.set_report_marker_locations(True)
    
    return ik_tool

# ======================= TRAJECTORY GENERATION =======================
def create_minimum_jerk_trajectory(start, target, duration, num_points=100):
    """ Generate smooth reaching trajectory """
    times = np.linspace(0, duration, num_points)
    trajectory = np.zeros((num_points, 3))
    
    for i in range(3):
        t_norm = times/duration
        trajectory[:, i] = start[i] + (target[i] - start[i]) * \
                          (10 * t_norm**3 - 15 * t_norm**4 + 6 * t_norm**5)
    
    return times, trajectory

# ======================= MAIN SIMULATION =======================
def run_complete_simulation():
    # 1. Load model (replace with your model)
    model_path = "W:\\OpenSim 4.5\\Resources\\Models\\Arm26\\arm26.osim"  # Should include hand body
    model = osim.Model(model_path)  # Should include hand body
    
    # 2. Define reaching parameters
    reaching_duration = 2.0
    start_pos = np.array([0.1, 0.1, 0.1])  # Near resting position
    target_pos = np.array([0.4, 0.5, 0.3])  # Reach target
    
    # 3. Generate desired trajectory
    times, hand_trajectory = create_minimum_jerk_trajectory(
        start_pos, target_pos, reaching_duration)
    
    # 4. Solve Inverse Kinematics
    print("Solving inverse kinematics...")
    ik_tool = setup_inverse_kinematics(model, "hand", hand_trajectory, times)
    ik_tool.run()
    
    # 5. Load IK results
    ik_results = osim.TimeSeriesTable("ik_results.mot")
    
    # 6. Setup controller to track IK solution
    controller = osim.PrescribedController()
    controller.setActuators(model.updActuators())
    
    # Map IK results to joint controls
    for j in range(model.getNumActuators()):
        actuator = model.getActuators().get(j)
        coord_name = actuator.getName().replace("_act", "")
        
        # Get joint angles from IK solution
        coord_values = ik_results.getDependentColumn(coord_name)
        
        # Create control function
        function = osim.PiecewiseLinearFunction()
        for i in range(ik_results.getNumRows()):
            t = ik_results.getIndependentColumn()[i]
            val = coord_values.get(i)
            function.addPoint(t, val)
        
        controller.prescribeControlForActuator(j, function)
    
    model.addController(controller)
    
    # 7. Run forward simulation
    print("Running forward dynamics...")
    state = model.initSystem()
    manager = osim.Manager(model)
    manager.setInitialTime(0)
    manager.setFinalTime(reaching_duration)
    manager.integrate(state)
    
    # 8. Analyze results
    analyze_results(model, manager, hand_trajectory, times)

# ======================= RESULT ANALYSIS =======================
def analyze_results(model, manager, desired_traj, times):
    """ Compare desired vs achieved trajectories """
    states_table = manager.getStatesTable()
    achieved_traj = np.zeros((len(times), 3))
    
    # Get achieved positions using forward kinematics
    for i, t in enumerate(times):
        state = states_table.getStateAtIndex(i)
        achieved_traj[i] = get_hand_position(model, state)
    
    # Plotting
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    ax.plot(desired_traj[:,0], desired_traj[:,1], desired_traj[:,2],
            'b-', label='Desired')
    ax.plot(achieved_traj[:,0], achieved_traj[:,1], achieved_traj[:,2],
            'r--', label='Achieved')
    
    ax.set_xlabel('X (m)'); ax.set_ylabel('Y (m)'); ax.set_zlabel('Z (m)')
    ax.set_title('Reaching Movement Trajectory')
    ax.legend()
    plt.show()
    
    # Calculate error
    rms_error = np.sqrt(np.mean(np.sum((desired_traj - achieved_traj)**2, axis=1)))
    print(f"Trajectory Tracking RMS Error: {rms_error:.4f} meters")

# Run the complete simulation
if __name__ == "__main__":
    run_complete_simulation()